In [59]:
import sys
from memory_profiler import profile

sys.path.append('/home/ak/Documents/Research/PaperCode/singlekernelclf/')
sys.path.append('/home/ak/Documents/Research/PaperCode/MultiKernelLearning')
import jsonpickle
from MKLpy.preprocessing import normalization, rescale_01
from MKLpy.metrics import pairwise
from MKLpy import generators
from MKLpy.algorithms import AverageMKL, EasyMKL
from sklearn.svm import SVC
from collections import OrderedDict
import os
import torch
import time
import pickle
import new_alternate_single_svm as nalsvm
from mkl_data_processing import storage_location, cross_validation_results_location, evaluate_predictions, oos_results_location, return_cross_val_symbol_path, model_dates_list
from MKLpy.multiclass import OneVsRestMKLClassifier, OneVsOneMKLClassifier
from MKLpy.metrics import margin, radius, ratio, trace, frobenius
from MKLpy.preprocessing import kernel_normalization
from mkl_model_cross_validation import load_pickled_in_filename
# evaluate the solution
from sklearn.metrics import accuracy_score, roc_auc_score
# MKL algorithms
from MKLpy.algorithms import AverageMKL, EasyMKL, \
    KOMD  # KOMD is not a MKL algorithm but a simple kernel machine like the SVM

In [60]:
if __name__ == '__main__':
    # pick the symbols you want to try out of sample
    # pick the model dates- this is important as you need to calculate the out of sample "forward" dates
    # pick label
    alternate_label = 'LabelsAlternateThree'
    # index of label - this can be redundant
    alternate_label_idx = list(nalsvm.labels_pickle_files).index(alternate_label)
    print(alternate_label)
    # clean data location
    for symbol in ['SHP.L']:
        clean_data_location = storage_location(symbol)
        print(symbol)
        # model dates list
        model_dates = model_dates_list(return_cross_val_symbol_path(symbol))
        # location of data -->dataDrive, Clean Data Storage, and label.
        pkl_file = load_pickled_in_filename(
            os.path.join(clean_data_location, os.listdir(clean_data_location)[alternate_label_idx]))
        date_keys = list(pkl_file.keys())
        model_date = '20170704'
        # list of out of sample dates
        for model_date in model_dates:
            start = time.time()
            forward_dates = nalsvm.forwardDates(date_keys, model_date)

            print('---------------> Doing Model Date:', model_date)

             # put the features in a tensor format
            Xtr = rescale_01(torch.Tensor(pkl_file[model_date][0].values))
            Xtr = normalization(Xtr)  # fitting model
            # put the labels in a tensor format
            Ytr = torch.Tensor(pkl_file[model_date][1].values)
            KLtr = generators.RBF_generator(Xte, gamma=[.001, .01, .1])
            deg = 5
            KLtr_poly = [pairwise.homogeneous_polynomial_kernel(Xtr, degree=d) for d in range(11)]
            
            K = KLtr_poly[deg]					#the HPK with degree 5
            K = kernel_normalization(K)		#normalize the kernel K (useless in the case of HPK computed on normalized data)

            score_margin = margin(K,Ytr)	#the distance between the positive and negative classes in the kernel space
            score_radius = radius(K)		#the radius of the Einimum Enclosing Ball containing data in the kernel space
            score_ratio  = ratio (K,Ytr)	#the radius/margin ratio defined as (radius**2/margin**2)/n_examples
            #the ratio can be also computed as score_radius**2/score_margin**2/len(Ytr)
            score_trace  = trace (K)		#the trace of the kernel matrix
            score_froben = frobenius(K)		#the Frobenius norm of a kernel matrix
            print ('done')
            print ('results of the %d-degree HP kernel:' % deg)
            print ('margin: %.4f, radius: %.4f, radiu-margin ratio: %.4f,' % (score_margin, score_radius, score_ratio))
            print ('trace: %.4f, frobenius norm: %.4f' % (score_trace, score_froben))





LabelsAlternateThree
SHP.L
/media/ak/My Passport/Data/FinDataReal/JointLocationsAlternateDataClean/CV_Results/SHP.L
---------------> Doing Model Date: 20170704
done
results of the 5-degree HP kernel:
margin: 0.0123, radius: 0.9220, radiu-margin ratio: 1.4698,
trace: 3822.0000, frobenius norm: 2260.2243
---------------> Doing Model Date: 20170705
done
results of the 5-degree HP kernel:
margin: 0.0033, radius: 0.9250, radiu-margin ratio: 12.3404,
trace: 6342.0000, frobenius norm: 3766.8599
---------------> Doing Model Date: 20170706


KeyboardInterrupt: 

In [48]:
# force garbage collect
nalsvm.gc.collect()
# kernels
KLrbf = generators.RBF_generator(Xtr, gamma=[.001, .01, .1])
# dont need the next bit
print('done with kernel')

# base learner- use c =1 or 10
# the c and lambda values need to be picked up by the cross-val results !
base_learner = SVC(C=1)
clf = AverageMKL().fit(KLrbf,Ytr)	#a wrapper for averaging kernels
print ('done')
K_average = clf.solution.ker_matrix	#the combined kernel matrix

done with kernel
done


In [49]:
K_average

tensor([[1.0000, 0.9931, 0.9854,  ..., 0.9391, 0.9431, 0.9399],
        [0.9931, 1.0000, 0.9849,  ..., 0.9382, 0.9423, 0.9390],
        [0.9854, 0.9849, 1.0000,  ..., 0.9471, 0.9500, 0.9474],
        ...,
        [0.9391, 0.9382, 0.9471,  ..., 1.0000, 0.9969, 0.9993],
        [0.9431, 0.9423, 0.9500,  ..., 0.9969, 1.0000, 0.9985],
        [0.9399, 0.9390, 0.9474,  ..., 0.9993, 0.9985, 1.0000]],
       dtype=torch.float64)

In [50]:
clf.decision_function(KL=KLrbf)

array([-2.47127334, -2.1812471 , -2.65054211, ..., -1.50867855,
        0.12374728, -0.92786341])

In [51]:
from sklearn.metrics import accuracy_score, roc_auc_score
import numpy as np
from MKLpy.multiclass import OneVsRestMKLClassifier, OneVsOneMKLClassifier


for forward_date in forward_dates:
    print('--------------')
    print(Xtr.shape)
    print(Ytr.shape)
    Xte = rescale_01(pkl_file[forward_date][0].values)
    Xte = normalization(Xte)
    Yte = pkl_file[forward_date][1].values
    print(Xte.shape)
    print(Yte.shape)

    y_pred = clf.predict(KLte)					#predictions
    y_score = clf.decision_function(KLte)		#rank
    accuracy = accuracy_score(Yte, y_pred)
    print ('Accuracy score: %.3f' % (accuracy))
    print('--------------')

--------------
torch.Size([3822, 26])
(3822,)
torch.Size([6342, 26])
(6342,)


ValueError: X.shape[1] = 6342 should be equal to 3822, the number of samples at training time

In [23]:

        print('Current Forward Date:', date)
        nalsvm.logmemoryusage("Before garbage collect")
        Xte = normalization(rescale_01(torch.Tensor(pkl_file[date][0].values)))
        Yte = torch.Tensor(pkl_file[date][1].values)
        print('got the test data')
 
        KLte = generators.RBF_generator(Xte, gamma=[.001, .01, .1])
        print('sorted out test dates bit done')
        nalsvm.gc.collect()
        y_pred = clf.predict(KLte)  # predictions
        y_score = clf.decision_function(KLte)  # rank
        print('finished predictions')
        # oos_svc_predictions = defaultdict(dict)
        accuracy = accuracy_score(Yte, y_pred)
        print('Accuracy score: %.3f' % accuracy)
#         except (ValueError, TypeError, EOFError, IndexError):
#             continue


Current Forward Date: 20170705
Memory usage at Before garbage collect is 1469.8782720000002mb.
got the test data
sorted out test dates bit done


ValueError: X.shape[1] = 6342 should be equal to 3822, the number of samples at training time

In [ ]:
#                 # force garbage collect
#                 nalsvm.gc.collect()
#                 # kernels
#                 KLrbf = generators.RBF_generator(Xtr, gamma=[.001, .01, .1])
#                 # dont need the next bit
#                 print('done with kernel')
#                 print(forward_dates)
#                 # base learner- use c =1 or 10
#                 # the c and lambda values need to be picked up by the cross-val results !
#                 base_learner = SVC(C=1)
#                 clf = EasyMKL(lam=0.1, multiclass_strategy='ova', learner=base_learner).fit(KLrbf, Ytr)
#                 mkl_avg = AverageMKL().fit(KLrbf, Ytr)
#                 print('done')
#                 print('the combination weights are:')
#                 # # this bit may be redundant here and we can put it somewhere else
#                 for sol in clf.solution:
#                     print('(%d vs all): ' % sol, clf.solution[sol].weights) #dont need this loop- can make it redundant in another file
#                 for date in forward_dates:
#                     print('Current Forward Date:', date)
#                     nalsvm.logmemoryusage("Before garbage collect")
#                     Xte = normalization(rescale_01(torch.Tensor(pkl_file[date][0].values)))
#                     Yte = torch.Tensor(pkl_file[date][1].values)
#                     print('got the test data')
#                     try:
#                         KLte = generators.RBF_generator(Xte, gamma=[.001, .01, .1])
#                         print('sorted out test dates bit done')
#                         nalsvm.gc.collect()
#                         y_pred = clf.predict(KLte)  # predictions
#                         y_score = clf.decision_function(KLte)  # rank
#                         print('finished predictions')
#                         # oos_svc_predictions = defaultdict(dict)
#                         accuracy = accuracy_score(Yte, y_pred)
#                         print('Accuracy score: %.3f' % accuracy)
#                         # create a dictionary to store all the results
#                         oos_results_dict = OrderedDict(dict)
#                         oos_score_dict = OrderedDict(dict)
#                         # store the results of the MKL
#                         oos_results_dict['Weighted MKL'][(model_date, date)] = evaluate_predictions(Yte, y_pred)
#                         oos_score_dict['Weighted MKL'][(model_date, date)] = y_score
#                         # average kernel as a base line
#                         y_preds_average = mkl_avg.predict(KLte)  # predict the output class
#                         y_scores_average = mkl_avg.decision_function(KLte)  # returns the projection on the distance vector
#                         average_accuracy = accuracy_score(Yte, y_preds_average)
#                         print ('Accuracy score: %.3f' % average_accuracy)
#                         # now store the results of the average kernel base-line
#                         oos_results_dict['Average KL'][(model_date, date)] =  evaluate_predictions(Yte, y_preds_average)
#                         oos_score_dict['Average KL'][(model_date, date)] = y_scores_average
#                     except (ValueError, TypeError, EOFError, IndexError):
#                         continue
#                     accuracy_file_name = "_".join((symbol, model_date, date,alternate_label,'OOSResult.pkl'))
#                     print(accuracy_file_name)
#                     pickle_dump_obj_to_filename(oos_results_location(symbol), symbol, accuracy_file_name, oos_results_dict)
#                     # pickle_dump_obj_to_filename(destinationPath, symbol, fileName, obj)

#             except (ValueError, TypeError, EOFError, IndexError):
#                 # at some point for clarity we need to clean these error up.
#                 continue
#                 nalsvm.gc.collect()
#                 print('done too')
#                 end = time.time()
#                 print(f'it took {end - start} seconds!')
